This notebook fetches data and creates visualization of 311 illegal dumping complaints in the vicinity of NYCHA properties. 

In [18]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium

from shapely import wkt
from urllib.parse import urlencode

from folium.plugins import MarkerCluster

In [6]:
API_devs = 'https://data.cityofnewyork.us/resource/5j2e-zhmb.csv?'
API_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.csv?'

In [12]:
df_dev = pd.read_csv(API_devs)

In [13]:
df_dev['the_geom'] = df_dev['the_geom'].apply(wkt.loads)
gdf_dev = gpd.GeoDataFrame(df_dev,geometry = df_dev.the_geom)

In [14]:
gdf_dev = gdf_dev.drop(columns='geometry')
gdf_dev = gdf_dev.rename(columns={'the_geom':'geometry',
                          'developmen':'Name'})

In [15]:
# Use project tracker to limit devs to current devs
df_project = pd.read_excel('/Users/shen/Desktop/Waste_IAPs_v2/DATA/Project Tracker.xlsx', sheet_name = "Overall Data -Updated May'21")
df_project = df_project[df_project['RAD Status'] != 'Converted'].reset_index(drop = True)
# remove private management
df_project = df_project[df_project['Management Jurisdiction (NYCHA Borough Group)'].str.lower() != 'private mgmt'].reset_index(drop = True)
df_project = df_project[['TDS #', 'Consolidated Name', 'Consolidated TDS #']]
df_project = df_project.rename(columns={'TDS #':'tds_num', 'Consolidated Name':'Consolidation', 'Consolidated TDS #':'cons_tds'})

In [16]:
gdf_dev = gdf_dev.merge(df_project, how = 'right', on = 'tds_num')
gdf_dev4326 = gdf_dev.set_crs(4326)

In [ ]:
gdf_dev4326.plot()

In [33]:
# project to NAD83 USft ny state plane - epsg 2263 
gdf_dev2263 = gdf_dev4326.to_crs(2263)

In [ ]:
gdf_dev2263.plot()

In [44]:
gdf_dev2263.head()

,geometry,Name,borough,tds_num,Consolidation,cons_tds
0,"MULTIPOLYGON (((1017198.821 245514.921, 101726...",1010 EAST 178TH STREET,BRONX,180,1010 EAST 178TH STREET,180
1,"MULTIPOLYGON (((1014152.363 249239.584, 101403...",EAST 180TH STREET-MONTEREY AVENUE,BRONX,208,1010 EAST 178TH STREET,180
2,"MULTIPOLYGON (((1015491.437 247805.798, 101543...",TWIN PARKS EAST (SITE 9),BRONX,287,1010 EAST 178TH STREET,180
3,"MULTIPOLYGON (((1010654.154 237046.324, 101071...",ADAMS,BRONX,118,Adams,118
4,"MULTIPOLYGON (((1001913.412 184970.656, 100187...",ALBANY,BROOKLYN,31,Albany,31


In [45]:
# Create a buffer around each polygon
buffer_2263 = gdf_dev2263.copy()
buffer_2263['geometry'] = gdf_dev2263.buffer(264)


In [49]:
buffer_shape_2263 = buffer_2263.dissolve(by = buffer_2263.cons_tds)

In [50]:
buffer_shape_4326 = buffer_shape_2263.to_crs(4326)

In [ ]:
buffer.plot()

In [35]:
buffer_4326 = buffer.to_crs(4326)

In [4]:
# Get 311
query_311 = {'$select': '*',
             '$where': 'created_date>="2020-07-01" and created_date<"2021-07-01" and descriptor="E1 Improper Disposal"',
             '$limit': 10000000} # if you don't specify 'limit', it returns 1000 rows as default

df_311 = pd.read_csv(API_311 + urlencode(query_311),
                 parse_dates = ['created_date'],
                 dtype = {'incident_zip':'str'})
df_311.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location
0,50454536,2021-05-03 10:35:00,2021-05-05T12:00:00.000,DSNY,A - Bronx,Dirty Conditions,E1 Improper Disposal,Sidewalk,10452,1037 OGDEN AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.833931,-73.928824,"\n, \n(40.83393144965909, -73.92882432434307)"
1,50467863,2021-05-04 10:31:00,2021-05-05T12:00:00.000,DSNY,A - Brooklyn,Dirty Conditions,E1 Improper Disposal,Sidewalk,11210,719 EAST 31 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.633503,-73.946735,"\n, \n(40.63350277324568, -73.94673533840827)"
2,50471453,2021-05-04 11:42:00,2021-05-05T12:00:00.000,DSNY,A - Staten Island,Dirty Conditions,E1 Improper Disposal,Sidewalk,10305,18 FATHER CAPODANNO BOULEVARD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.596092,-74.061988,"\n, \n(40.596092349152194, -74.06198846254166)"
3,50471542,2021-05-04 10:05:00,2021-05-05T12:00:00.000,DSNY,A - Brooklyn,Dirty Conditions,E1 Improper Disposal,Sidewalk,11210,642 EAST 31 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.635306,-73.946946,"\n, \n(40.6353061966652, -73.94694647338598)"
4,50473805,2021-05-03 15:19:00,2021-05-05T12:00:00.000,DSNY,A - Bronx,Dirty Conditions,E1 Improper Disposal,Sidewalk,10457,2144 PROSPECT AVENUE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.848312,-73.885637,"\n, \n(40.848311931775356, -73.88563740719708)"


In [5]:
len(df_311)

4746

In [25]:
gdf_311_4326 = gpd.GeoDataFrame(df_311, geometry=gpd.points_from_xy(df_311.longitude, df_311.latitude)).set_crs(4326)

In [27]:
gdf_311_4326 = gdf_311_4326.dropna(subset = ['latitude', 'longitude'])

In [53]:
gdf_311_inbuffer_4326 = gpd.sjoin(gdf_311_4326, buffer_shape_4326, op='within') 

In [54]:
gdf_311_inbuffer_4326.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,...,latitude,longitude,location,geometry,index_right,Name,borough_right,tds_num,Consolidation,cons_tds
10,50690124,2021-05-26 08:55:00,2021-05-29T12:00:00.000,DSNY,A - Manhattan,Dirty Conditions,E1 Improper Disposal,Sidewalk,10001,450 WEST 25 STREET,...,40.749191,-74.003786,"\n, \n(40.749190587884065, -74.00378596163142)",POINT (-74.00379 40.74919),134,CHELSEA,MANHATTAN,134,Chelsea,134
1380,50768079,2021-06-03 17:00:00,2021-06-04T12:00:00.000,DSNY,A - Manhattan,Dirty Conditions,E1 Improper Disposal,Sidewalk,10011,235 10 AVENUE,...,40.748628,-74.003508,"\n, \n(40.74862792332135, -74.00350803014729)",POINT (-74.00351 40.74863),134,CHELSEA,MANHATTAN,134,Chelsea,134
23,51062093,2021-06-30 11:32:00,2021-07-01T12:00:00.000,DSNY,A - Manhattan,Dirty Conditions,E1 Improper Disposal,Sidewalk,10029,324 EAST 116 STREET,...,40.796655,-73.936969,"\n, \n(40.796654645095565, -73.9369688110391)",POINT (-73.93697 40.79665),64,CORSI HOUSES,MANHATTAN,199,Jefferson,64
1651,46718803,2020-07-01 15:16:00,2020-07-28T12:00:00.000,DSNY,A - Manhattan,Dirty Conditions,E1 Improper Disposal,Sidewalk,10029,339 EAST 115 STREET,...,40.795650,-73.936703,"\n, \n(40.79564992822559, -73.93670250136257)",POINT (-73.93670 40.79565),64,CORSI HOUSES,MANHATTAN,199,Jefferson,64
4405,49940122,2021-03-04 16:43:00,2021-03-08T12:00:00.000,DSNY,A - Manhattan,Dirty Conditions,E1 Improper Disposal,Sidewalk,10029,204 EAST 112 STREET,...,40.795312,-73.941676,"\n, \n(40.79531220567817, -73.94167601812121)",POINT (-73.94168 40.79531),64,CORSI HOUSES,MANHATTAN,199,Jefferson,64


In [58]:
gdf_311_inbuffer_4326 = gdf_311_inbuffer_4326.drop_duplicates(subset='geometry')

In [66]:
base_map = folium.Map(location=[40.693943, -73.985880], zoom_start = 11,
                     tiles='cartodbpositron')
style_dev = {'weight':0,
        'fillColor':'#800020',
        'color': '#800020'}
style_buffer = {'weight':0,
        'fillColor':'gray',
        'color': 'gray'}

folium.GeoJson(buffer_shape_4326, zoom_on_click= True, style_function=lambda x: style_buffer).add_to(base_map)
folium.GeoJson(gdf_dev4326, style_function=lambda x: style_dev).add_to(base_map)

# popup=folium.GeoJsonPopup(fields=['Consolidation'])

marker_cluster = MarkerCluster().add_to(base_map)

for row in gdf_311_inbuffer_4326.itertuples():
    coordinates = [row.latitude, row.longitude]

    #html = f'''{row.Consolidation} Consolidation<br>
    #         {row.Name.title()} Development'''

    popup = f'{row.created_date}'

    # tooltip=html, 
    folium.Marker(location = coordinates, popup = popup,icon = folium.Icon(color='#800020')).add_to(marker_cluster)

/Users/shen/Library/Python/3.6/lib/python/site-packages/ipykernel_launcher.py:26: UserWarning: color argument of Icon should be one of: {'red', 'cadetblue', 'darkpurple', 'lightgreen', 'green', 'white', 'gray', 'beige', 'purple', 'pink', 'black', 'lightblue', 'blue', 'lightgray', 'darkgreen', 'darkblue', 'orange', 'darkred', 'lightred'}.


In [67]:
base_map

In [68]:
base_map.save(outfile = 'docs/test.html')